# Import libs and load dfs

In [ ]:
# Classic imports
import pandas as pd
import numpy as np

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Other imports
import tqdm

In [ ]:
# Load the datasets

# folder = "PRAMA2025/"
# modify_df = pd.read_csv(folder + 'train_with_clusters.csv')
# test_df = pd.read_csv(folder + 'test_with_clusters.csv')

# for those who have the datasets in the same folder as the python code (which is the way to go ngl)
# florian is quite a flexer but we don't have to be like him

origin_train_name = "train_data"
origin_test_name = "test_data"

train_df = pd.read_csv('CSV DATA/'+origin_train_name+'.csv')  # fichier csv VIERGE d'origine du train
test_df = pd.read_csv('CSV DATA/'+origin_test_name+'.csv')  # fichier csv VIERGE d'origine du test

#little printy print 
print("Train shape: ", modify_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (17147, 21)
Test shape:  (4287, 20)


# Transforming the data

### Attention nouvelle règle : 

- Damien is to come to the next classes with no underwear and a kilt.
- We'll add a value between 0 and 1 showcasing the evolution of the year : 0 for the beginnin of the year, 1 for the end of the year. Then, we'll multiply this by pi and use the sine of this value.
- This way we'll have values closer to 1 for 'colder' days and 0 for 'hotter' days. 

## Sinus of all date data

In [ ]:
def sin_ratios_for_date(modify_df, name):
    # Preprocess the datasets
    # Convert the Sales_Date to datetime format
    modify_df['date'] = pd.to_datetime(modify_df['date'])

    # Extract useful date features (e.g., year, month, day)
    modify_df['Year'] = modify_df['date'].dt.year
    modify_df['Month'] = modify_df['date'].dt.month
    modify_df['Day'] = modify_df['date'].dt.day
    modify_df['sin_month'] = modify_df['Month'].map(lambda x: np.sin(float(x)/12))

    modify_df['day_of_year'] = modify_df['date'].dt.dayofyear
    # Calcul du nombre de jours dans l'année (365 ou 366)
    modify_df['days_in_year'] = np.where(modify_df['date'].dt.is_leap_year, 366, 365)
    # Calcul El Famoso ratio (float entre 0 et 1)
    modify_df['day_ratio'] = modify_df['day_of_year'] / modify_df['days_in_year']
    # Calculer le sinus de pi * ratio
    modify_df['sin_dayofyear'] = np.sin(np.pi * modify_df['day_ratio'])
    modify_df.drop(columns=['day_of_year', 'days_in_year', 'day_ratio'], inplace=True)
    modify_df = modify_df.drop(columns=['date', 'Year', 'Month', 'Day'])

    new_name = name + "_with_sin_ratios"

    return modify_df, new_name

## Transform df to have price/square meters

In [ ]:
def create_new_df_with_m2(to_modify_df, name):
    # create a new data frame with everything plus a column corresponding to 1 if "m2_jardin" is >0, 0 otherwise
    # we will also do the same for m2_etage and m2_soussol
    modify_df = to_modify_df.copy()
    modify_df['jardin'] = train_df['m2_jardin'].map(lambda x: 1 if x > 0 else 0)
    modify_df['etage'] = train_df['m2_etage'].map(lambda x: 1 if x > 0 else 0)
    modify_df['soussol'] = train_df['m2_soussol'].map(lambda x: 1 if x > 0 else 0)

    # now add three new columns per df, one for interior m2 and one for exterior m2 and one for total m2
    # for interior we already have m2_interieur which is the sum of etage and soussol
    modify_df['m2_outside'] = modify_df['m2_jardin']
    modify_df['m2_total'] = modify_df['m2_interieur'] + modify_df['m2_outside']

    # now we add two columns per df, one for prix per m2 interior and one for prix per m2 total
    modify_df['prix_m2_interieur'] = modify_df['prix'] / modify_df['m2_interieur']
    modify_df['prix_m2_total'] = modify_df['prix'] / modify_df['m2_total']

    # now export
    train_name = "train_with_clusters_and_m2.csv"
    modify_df.to_csv(train_name, index=False)

    print("Train shape: ", modify_df.shape)

    new_name = name + "_with_m2_price"

    return modify_df, new_name

## Scaling the data

In [ ]:
def scaling_data(df, name, scaler = None):
    if scaler :
        scaler = scaler
    else :
        # Initialize the MinMaxScaler
        scaler = MinMaxScaler()

    # Scale the data
    to_scale1 = df.copy()  # to not modify the original one
    scaled_df1 = pd.DataFrame(scaler.fit_transform(to_scale1), columns=to_scale1.columns)

    scaled_name = name + "_scaled"

    return scaler, scaled_df1, scaled_name

## Export the csv

In [ ]:
def exporting_df(df, name):
    df.to_csv('CSV data/'+ name + '.csv', index=False)

### Call the transfos

In [ ]:
# dates to sinus function
train_sin, train_sin_name = sin_ratios_for_date(train_df, origin_train_name)
test_sin, test_sin_name = sin_ratios_for_date(test_df, origin_test_name)

# find price per square meter
train_sin_m2, train_sin_m2_name = create_new_df_with_m2(train_sin, train_sin_name)

# Scale the data
scaler1, train_sin_m2_scaled, train_sin_m2_scaled_name = scaling_data(train_sin_m2, train_sin_m2_name)
scaler1, train_sin_scaled, train_sin_scaled_name = scaling_data(train_sin, train_sin_name, scaler=scaler1)
scaler1, test_sin_scaled, test_sin_scaled_name = scaling_data(test_sin, test_sin_name, scaler=scaler1)

In [ ]:
train_sin_m2_scaled.head()

,id,prix,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,...,Month,Day,cos_month,jardin,etage,soussol,m2_outside,m2_total,prix_m2_interieur,prix_m2_total
0,4443800785,481000,2,1.00,150.501672,360.460795,85.470085,65.031587,1.0,0,...,11,21,0.004426,1,1,1,360.460795,510.962467,3195.977778,941.360727
1,2722059013,550000,2,1.00,117.985879,4046.822742,117.985879,0.000000,1.0,0,...,2,4,-0.416147,1,1,0,4046.822742,4164.808621,4661.574803,132.058889
2,8856004730,199950,2,2.75,147.714604,1943.236715,147.714604,0.000000,1.5,0,...,9,17,-0.911130,1,1,0,1943.236715,2090.951319,1353.623774,95.626330
3,2473003210,364808,3,1.75,215.533259,731.605351,150.501672,65.031587,1.0,0,...,3,13,-0.989992,1,1,1,731.605351,947.138610,1692.583324,385.168545
4,6743700335,470000,3,2.00,167.224080,1176.978818,167.224080,0.000000,1.0,0,...,6,4,0.960170,1,1,0,1176.978818,1344.202899,2810.600000,349.649596


In [ ]:
exporting_df(train_sin_m2_scaled, train_sin_m2_scaled_name)

In [ ]:
# X_train, y_train, X_test, y_test = train_test_split(train_new.drop(columns=['prix']), train_new['prix'], test_size=0.2, random_state=42)

,id,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,vue_note,...,cluster_tres_eleve,Year,Month,Day,cos_month,jardin,etage,soussol,m2_outside,m2_total
0,3600072,4,2.75,206.243032,493.311037,108.695652,97.547380,1.0,0,0,...,0,2015,3,30,-0.989992,1,1,1,493.311037,699.554069
1,6200017,3,1.00,124.489038,1982.162765,124.489038,0.000000,1.5,0,0,...,0,2014,11,12,0.004426,1,1,0,1982.162765,2106.651802
2,7600136,2,2.00,104.979562,106.651802,74.321813,30.657748,2.0,0,0,...,0,2014,7,18,0.753902,1,1,1,106.651802,211.631364
3,11200400,3,2.50,177.443330,416.945373,177.443330,0.000000,2.0,0,0,...,0,2014,9,23,-0.911130,1,1,0,416.945373,594.388703
4,11500890,3,2.50,290.784095,812.894835,290.784095,0.000000,2.0,0,0,...,0,2015,3,12,-0.989992,1,1,0,812.894835,1103.678930


In [ ]:
# clusters = ['cluster_tres_bas', 'cluster_bas', 'cluster_moyen', 'cluster_eleve', 'cluster_tres_eleve']
# train_new.drop(columns=clusters, inplace=True)